# Блок сбора данных

## vkParserController
Место откуда собираем данные: __Vkontakte__

Парсер подготавливался как универсальный, поэтому собирает все данные без ограничений

### План:
1. Подготавливаем список групп
2. Проходим списку групп
    1. Для каждой группы смотрим количество постов
    2. Проходим в цикле и собираем по 100 постов за раз(ограничение выдачи api)
    3. Записываем пост в базу
3. Выбираем нужные посты(в данном случае где упомянается вирус Ковид/covid/ etc)
4. Проходим по всем постам(у которых нет отметки о сборе комметариев, что бы в случае проблем не проходить одни и те же посты несолько раз)
    1. Собираем комметарии по аналогии со сбором постов
    2. Записываем комментарий в базу
    3. Отмечаем пост в котором собрали комметарии
    

## dbControllerPsql
Для сбора и хранения ипользовал PostgreSQL 

Можно переписать контроллер для использования другого средства хранения. Была попытка записи в sqllite, но у нее есть проблема с большим потоком данных на запись

In [1]:
import import_ipynb
import time

from dbControllerPsql import DataBase
from vkParserController import vkParser

importing Jupyter notebook from dbControllerPsql.ipynb
importing Jupyter notebook from vkParserController.ipynb


In [3]:
DB=DataBase(port=54320)
DB.create_tables()
parser=vkParser(phone='', password='')

In [4]:
def post_parser(group_list,offset=0,limit=0):
    for group in group_list:
        count_posts=parser.getGroupPostsCount(group)
        print("Update {}. Total count {}".format(group,count_posts))
        if(limit == 0):
            limit=count_posts
            
        while(offset < limit):
            try:
                posts=parser.getGroupPosts(group,offset)
                DB.add_post(posts)
                offset=offset+100
                print("Update posts. Offset {}".format(offset))
            except Exception  as exept:
                print(exept)
        offset=0
        
        

In [22]:
def comment_parser(post_list):

    for post in post_list:
        try:
            count_posts=parser.getGroupCommentsCount(post[0],post[1])
            offset=0
            while(offset < count_posts):
                comments=parser.getPostComments(post[0],post[1],offset)
                DB.add_comments(comments)
                offset=offset+50
                DB.setUpdateFlag(post[0],post[1])
                print("Update post {}".format(post[0]))
        except Exception  as exp:
                print('Comment parse error')
                print(exp) 

## Блок для сбора постов


In [12]:
#список групп для сбора постов
group_list=['act54','incident_nsk','vk_moscow','this_is_sochi','ruscentrvietnam','piter','nhatrang_vietnam','russianthai','infomoscow24',
          'this_is_sochi','kzngo']


In [13]:
post_parser(group_list,limit=0)

Update inctsk. Total count 2617
Update posts. Offset 100
Update posts. Offset 200
Update posts. Offset 300
Update posts. Offset 400
Update posts. Offset 500
Update posts. Offset 600
Update posts. Offset 700
Update posts. Offset 800
Update posts. Offset 900
Update posts. Offset 1000
Update posts. Offset 1100
Update posts. Offset 1200
Update posts. Offset 1300
Update posts. Offset 1400
Update posts. Offset 1500
Update posts. Offset 1600
Update posts. Offset 1700
Update posts. Offset 1800
Update posts. Offset 1900
Update posts. Offset 2000
Update posts. Offset 2100
Update posts. Offset 2200
Update posts. Offset 2300
Update posts. Offset 2400
Update posts. Offset 2500
Update posts. Offset 2600
Update posts. Offset 2700


## Блок для сбора комментариев

Выбираем посты где есть упоминание вируса

In [23]:
#получаем список постов с 1 января
post_list=DB.getIdPosts(q="  (text ILIKE '%коронавирус%' OR text ILIKE '%короновирус%' OR text ILIKE '%covid%') and (update_flag=0)")

In [24]:
post_list

[(9314251, -31101527),
 (9611876, -31101527),
 (9387546, -31101527),
 (9393514, -31101527),
 (9377757, -31101527),
 (2281514, -68030706),
 (4447161, -20845272),
 (5458034, -99099155),
 (2330280, -68030706),
 (2221302, -68030706),
 (2266593, -68030706),
 (2216593, -68030706),
 (2209656, -68030706),
 (2121278, -68030706),
 (2117456, -68030706),
 (2026907, -68030706),
 (2082760, -68030706),
 (2004023, -68030706),
 (2014084, -68030706),
 (1958467, -68030706),
 (423446, -65139424),
 (424815, -65139424),
 (424523, -65139424),
 (140721, -19788021),
 (418451, -65139424),
 (416737, -65139424),
 (416691, -65139424),
 (160402, -45990745),
 (128777, -19788021),
 (170671, -45990745),
 (170997, -45990745),
 (170938, -45990745),
 (170935, -45990745),
 (170860, -45990745),
 (170859, -45990745),
 (170964, -45990745),
 (170890, -45990745),
 (170611, -45990745),
 (170597, -45990745),
 (170577, -45990745),
 (170575, -45990745),
 (170475, -45990745),
 (170525, -45990745),
 (170176, -45990745),
 (170462, -4

In [25]:
comment_parser(post_list)

Update post 9314251
Update post 9314251
Update post 9314251
Update post 9314251
Update post 9314251
Update post 9314251
Update post 9314251
Update post 9314251
Update post 6040704
Comment parse error
[15] Access denied: post was deleted
Update post 5030705
Update post 5030705
Update post 5030705
Update post 5030705
Update post 1573627
Update post 1573627
Update post 6042606
Update post 6042606
Update post 6042606
Update post 6042606
Update post 6042606
Update post 6042606
Update post 6042606
Update post 6042606
Update post 6028686
Update post 6028686
Update post 2354932
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9958998
Update post 9954872
Update post 9954872
Update post 16182
Update post 1565255
Updat

Update post 2353508
Update post 2352389
Update post 6060402
Update post 424359
Update post 423957
Update post 423137
Update post 7307372
Update post 7297018
Update post 7295502
Update post 7295329
Update post 7295329
Update post 152141
Update post 150524
Update post 150524
Update post 150524
Update post 172702
Update post 10008908
Update post 10008908
Update post 172511
Update post 172510
Update post 172395
Update post 172286
Update post 172277
Update post 172368
Update post 172360
Update post 10005859
Update post 172187
Update post 172066
Update post 171941
Update post 171865
Update post 171849
Update post 1567718
Update post 1567718
Update post 1567718
Update post 1567718
Update post 1567718
Update post 1567718
Update post 1567718
Update post 1567689
Update post 2359293
Update post 2356303
Update post 2356408
Update post 2355731
Update post 2354788
Update post 2355675
Update post 2355544
Update post 2355220
Update post 2354641
Update post 2352985
Update post 2354155
Update post 23526

Update post 5054022
Update post 5054022
Update post 5054022
Update post 5054022
Update post 5054022
Update post 5054022
Update post 5053821
Update post 5053821
Update post 5053821
Update post 5050538
Update post 5051889
Update post 5051889
Update post 5051889
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051440
Update post 5051364
Update post 5051364
Update post 5051364
Update post 5051364
Update post 5049456
Update post 5049456
Update post 5049456
Update post 5049456
Update post 5049456
Update post 5049456
Update post 5049456
Update post 5049456
Update post 5049456
Update post 5042585
Update post 5042585
Update post 5048188
Update post 5048188
Update post 5047445
Update post 5047445
Update post 5046277
Update post 5046277
Update post 5046836
Update post 5046836
Update post 5046002
Update post 5046002
Update post 5046002


Update post 10020424
Update post 10019776
Update post 16081
Update post 10018625
Update post 10018625
Update post 10018625
Update post 10018625
Update post 10018625
Update post 10018625
Update post 10018625
Update post 10017258
Update post 10013234
Update post 10013234
Update post 10010700
Update post 10009882
Update post 10009882
Update post 10009882
Update post 10009882
Update post 10003334
Update post 10003334
Update post 10003334
Update post 10003334
Update post 10011844
Update post 10001264
Update post 10001264
Update post 10001264
Update post 10001264
Update post 9996331
Update post 9996331
Update post 9996331
Update post 9996331
Update post 9996331
Update post 9996331
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9994329
Update post 9993460
Update post 9993460
Update post 9993

Для удобства презентации, все комметарии выгрузил в csv. Для работы можно использовать из базы.

База объемная, к презентации не прилагаю.